In [1]:
import numpy as np              # Python's standard numerical library
import matplotlib.pyplot as plt # Python's scientific visualization library
import pyidi                    # Python HSC data analysis library
import pickle as pk
import os
from pixel_setter2 import PixelSetter#, play_video, detect_peaks
from pixel_setter import play_video
from skimage import feature, measure
import importlib
from EMA_functions import *
import matplotlib.animation as animation
# from scipy.signal import convolve2d
from scipy.ndimage import generic_filter
root_drive_sim = os.path.normpath('G:/.shortcut-targets-by-id/1k1B8zPb3T8H7y6x0irFZnzzmfQPHMRPx/Illimited Lab Projects/Spiders/Simulations')
root = 'C:/Users/thijsmas/Documents/GitHub/pyidi_data/'

c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
directory = os.path.join(root_drive_sim, 'tp')
file_name_vec = os.listdir(directory)
file_name_vec = [file for file in file_name_vec if file.endswith('.txt')]

print(file_name_vec)

folder_paths = ['H:/My Drive/PHD/HSC', 'D:/HSC', 'F:/', 'E:/thijs/', 'C:/Users/thijs/Documents/HSC/', 'D:/thijsmas/HSC']

['tp_Full_web_ecc0_new_Floc9_v0.txt', 'tp_Full_web_ecc0_new_Floc9_v1.txt', 'tp_Full_web_ecc0_new_Floc3_v0.txt', 'tp_Full_web_ecc1_new_Floc1_v0.txt', 'tp_Full_web_ecc1_new_Floc1_v1.txt', 'tp_Full_web_ecc1_new_Floc1_v2.txt', 'tp_Full_web_ecc2_new_Floc7_v1.txt', 'tp_Full_web_ecc1_new_Floc1_v3.txt', 'tp_Full_web_ecc1_new_Floc1_v4.txt', 'tp_Full_web_ecc1_new_Floc1_v5.txt']


In [3]:
%matplotlib qt
roi_size = (9, 9)
reference_image = (0, 30)
# mraw_range = (1, 502)
for file_name in file_name_vec:
    file_name_video = file_name[3:-4]
    video = open_video(file_name_video, folder_paths)
    tp = np.loadtxt(os.path.join(root_drive_sim, 'tp', file_name), delimiter=',')
    video.set_points(tp)
    video.set_method('lk')
    video.method.configure(reference_image = reference_image, roi_size= roi_size) #, mraw_range = mraw_range
    print('start simulating: '  + file_name)
    d = video.get_displacements(processes = -1) 
    d = video.displacements
    print('finished simulating: '  + file_name) 

    path = os.path.join(root, file_name_video+f'_d_rs{str(roi_size)}_ri{str(reference_image)}'+'.pkl')
    with open(path, 'wb') as f:
        pk.dump({'displacement':d, 'tracking points': tp}, f)
    
    dd = np.diff(d, axis=1)
    last_10_zeros = np.any(dd[:, -10:, :] == 0, axis=(1,2))
    large_d = np.any(np.linalg.norm(d, axis=2) > 15, axis = 1)
    good_points = np.logical_not(last_10_zeros | large_d)

    first_frame = 1

    td = d +  tp.reshape(len(tp),1,2)
    ani = play_video(video, range(first_frame, 500), interval=30, points=td[good_points, first_frame:, :])
    writer = animation.FFMpegWriter(fps=60)
    print('start saving video: '  + file_name)
    ani.save(os.path.join(root_drive_sim,'anim', f'{file_name_video}_d_rs{str(roi_size)}_ri{str(reference_image)}_anim.mp4'), writer=writer)
    print('finished saving video: '  + file_name)
    plt.close('all')



NameError: name 'open_video' is not defined